In [1]:
# Dependencies
import numpy as np
import pandas as pd

In [2]:
import tensorflow
tensorflow.keras.__version__

'2.2.4-tf'

In [3]:
#read csv
diversity = pd.read_csv('ml_csv.csv')
diversity.head()

,company,ownership,year,valuations,eeo_1 status,sum(female),sum(male),sum(American_Indian_Alaskan_Native),sum(Asian),sum(Black_or_African_American),...,sum(Administrative support),sum(Craft workers),sum(Executives),sum(Managers),sum(Professionals),sum(Sales workers),sum(Service workers),sum(Technicians),sum(laborers and helpers),sum(operatives)
0,NetApp,public,2016,11000000000,YES,4522,14379,67,5184,633,...,182,0,72,2006,7764,1302,0,36,0,0
1,Lyft,private,2016,8000000000,YES,1431,1970,18,657,206,...,836,0,30,518,1366,112,4,0,0,0
2,Salesforce,public,2016,58000000000,YES,13189,28212,53,10000,968,...,830,0,474,5204,16238,6686,0,0,0,0
3,Hewlett Packard Enterprise,public,2016,39000000000,YES,34390,69588,412,13268,7468,...,5840,0,1198,11880,68246,8958,0,7354,366,136
4,Sanmina-SCI,public,2016,3000000000,YES,6280,9745,58,4021,944,...,1362,322,244,1182,2264,178,114,1416,8,3320


## Data Pre-Processing

In [4]:
#drop unimportant columns
diversity_without_year=diversity.drop("year",axis=1)
diversity_without_eeo=diversity_without_year.drop("eeo_1 status",axis=1)
diversity_without_company=diversity_without_eeo.drop("company",axis=1)
diversity_without_company.head()

,ownership,valuations,sum(female),sum(male),sum(American_Indian_Alaskan_Native),sum(Asian),sum(Black_or_African_American),sum(Hispanic_or_Latino),sum(Native_Hawaiian_or_Pacific_Islander),sum(Overall_totals),...,sum(Administrative support),sum(Craft workers),sum(Executives),sum(Managers),sum(Professionals),sum(Sales workers),sum(Service workers),sum(Technicians),sum(laborers and helpers),sum(operatives)
0,public,11000000000,4522,14379,67,5184,633,774,26,18901,...,182,0,72,2006,7764,1302,0,36,0,0
1,private,8000000000,1431,1970,18,657,206,241,9,3401,...,836,0,30,518,1366,112,4,0,0,0
2,public,58000000000,13189,28212,53,10000,968,1656,160,41401,...,830,0,474,5204,16238,6686,0,0,0,0
3,public,39000000000,34390,69588,412,13268,7468,5974,160,103978,...,5840,0,1198,11880,68246,8958,0,7354,366,136
4,public,3000000000,6280,9745,58,4021,944,1825,34,16025,...,1362,322,244,1182,2264,178,114,1416,8,3320


In [5]:
#split data into X and y
X = diversity_without_company.drop("ownership", axis=1)
y = diversity_without_company["ownership"]
print(X.shape, y.shape)

(25, 21) (25,)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
print(X_train.shape,y_train.shape)

(18, 21) (18,)


In [20]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [22]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Model

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [24]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=21))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [25]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               2200      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 12,502
Trainable params: 12,502
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Train on 18 samples
Epoch 1/60
18/18 - 1s - loss: 0.6282 - accuracy: 0.6667
Epoch 2/60
18/18 - 0s - loss: 0.6070 - accuracy: 0.6667
Epoch 3/60
18/18 - 0s - loss: 0.5882 - accuracy: 0.6667
Epoch 4/60
18/18 - 0s - loss: 0.5717 - accuracy: 0.6667
Epoch 5/60
18/18 - 0s - loss: 0.5569 - accuracy: 0.6667
Epoch 6/60
18/18 - 0s - loss: 0.5435 - accuracy: 0.6667
Epoch 7/60
18/18 - 0s - loss: 0.5314 - accuracy: 0.6667
Epoch 8/60
18/18 - 0s - loss: 0.5204 - accuracy: 0.6667
Epoch 9/60
18/18 - 0s - loss: 0.5105 - accuracy: 0.6667
Epoch 10/60
18/18 - 0s - loss: 0.5014 - accuracy: 0.6667
Epoch 11/60
18/18 - 0s - loss: 0.4932 - accuracy: 0.6667
Epoch 12/60
18/18 - 0s - loss: 0.4857 - accuracy: 0.6667
Epoch 13/60
18/18 - 0s - loss: 0.4788 - accuracy: 0.6667
Epoch 14/60
18/18 - 0s - loss: 0.4725 - accuracy: 0.6667
Epoch 15/60
18/18 - 0s - loss: 0.4666 - accuracy: 0.6667
Epoch 16/60
18/18 - 0s - loss: 0.4613 - accuracy: 0.6667
Epoch 17/60
18/18 - 0s - loss: 0.4562 - accuracy: 0.6667
Epoch 18/60
18/18 - 

## Quantify our Trained Model

In [28]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

7/7 - 0s - loss: 0.2114 - accuracy: 0.8571
Normal Neural Network - Loss: 0.21143092215061188, Accuracy: 0.8571428656578064


## Make Predictions

In [29]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [30]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['public' 'public' 'public' 'public' 'public']
Actual Labels: ['public', 'public', 'public', 'public', 'public']
